# Machine Learning - Image Classification

In [1]:
#importing
import os

import torch
import torch.nn as nn

from dotenv import load_dotenv
from sklearn.model_selection import train_test_split
from torch import optim

from utils import *

## Database importing

In [2]:
# TODO: pouzi toto martin
load_dotenv()
DAT_PATH = os.getenv("TRAIN_DATASET_PATH")

ANIMALS_DATAFRAME = load_dataset_info("../data/archive/raw-img")
ANIMALS_DATAFRAME.head()

,filepath,label,format,color_mode,bit_depth,width,height,area,aspect_ratio,brightness
0,..\data\archive\raw-img\cane\OIF-e2bexWrojgtQn...,cane,.jpeg,RGB,8,300,225,67500,1.333333,162.620696
1,..\data\archive\raw-img\cane\OIP---A27bIBcUgX1...,cane,.jpeg,RGB,8,300,214,64200,1.401869,171.896558
2,..\data\archive\raw-img\cane\OIP---cByAiEbIxIA...,cane,.jpeg,RGB,8,153,300,45900,0.510000,121.562360
3,..\data\archive\raw-img\cane\OIP---ZIdwfUcJeVx...,cane,.jpeg,RGB,8,300,225,67500,1.333333,121.487343
4,..\data\archive\raw-img\cane\OIP---ZRsOF7zsMqh...,cane,.jpeg,RGB,8,300,225,67500,1.333333,113.152356


In [3]:
train_df, test_df = train_test_split(ANIMALS_DATAFRAME, test_size=0.05, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.0526, random_state=42)

In [4]:
#setting up dataloaders

TARGET_SIZE = (256, 256)
MAX_SIZE = 500
BATCH_SIZE = 42
NUM_CLASSES = ANIMALS_DATAFRAME.label.nunique()
CLASS_LABELS = {name: idx for idx, name in enumerate(np.sort(ANIMALS_DATAFRAME.label.unique()))}

# vypocet frekvencie augmentacie -> aby nebol model biasnuty iba na majoritne categorie obrazkov
counts = train_df.label.value_counts()
max_count = counts.max()
aug_strength = (max_count / counts).to_dict()

print(aug_strength)

train_gen = AnimalImageGenerator(
    df=train_df,
    batch_size=BATCH_SIZE,
    target_size=TARGET_SIZE,
    num_classes=NUM_CLASSES,
    augment=True,
    shuffle=True,
    aug_strength=aug_strength,
    max_size=MAX_SIZE,
    class_mapping=CLASS_LABELS,
)

test_gen = AnimalImageGenerator(
    df=test_df,
    batch_size=BATCH_SIZE,
    target_size=TARGET_SIZE,
    num_classes=NUM_CLASSES,
    augment=False,
    shuffle=False,
    max_size=MAX_SIZE,
    class_mapping=CLASS_LABELS,
)

val_gen = AnimalImageGenerator(
    df=val_df,
    batch_size=BATCH_SIZE,
    target_size=TARGET_SIZE,
    num_classes=NUM_CLASSES,
    augment=False,
    shuffle=False,
    max_size=MAX_SIZE,
    class_mapping=CLASS_LABELS,
)


{'cane': 1.0, 'ragno': 1.013622719926114, 'gallina': 1.5667380442541041, 'cavallo': 1.8688803746275011, 'farfalla': 2.32521186440678, 'mucca': 2.564252336448598, 'scoiattolo': 2.642986152919928, 'pecora': 2.670316301703163, 'gatto': 2.9344919786096257, 'elefante': 3.408385093167702}
---
Total images: 23561
Num classes: 10
---
---
Total images: 1309
Num classes: 10
---
---
Total images: 1309
Num classes: 10
---


In [5]:
#testovaci vypis
images, labels = train_gen[0]     # zavolá __getitem__(0) pre prvy batch
print("Images shape:", images.shape)
print("Labels shape:", labels.shape)
print("Image dtype:", images.dtype)
print("Label sample:", labels[0])

Images shape: (56, 256, 256, 3)
Labels shape: (56,)
Image dtype: uint8
Label sample: 0


## Model implemetation

In [6]:
import torch.nn.functional as fun

In [7]:
#PRE TUTO FUNKCIU NEGENERUJ ZIADNE KOMENTARE
class ImageClassifier(nn.Module):
    def __init__(self, classes: int):
        super(ImageClassifier, self).__init__()
        self.numberOfClasses = classes

        # Convolution layres ONLY WORKS with RGB because of in_channels, kernel_size for filtering is 3 stride 1 padding 1 for size preservation
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1,padding=1)
        self.bn1 = nn.BatchNorm2d(16)

        # Significant for grad-CAM
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1,padding=1)
        self.bn2 = nn.BatchNorm2d(32)

        #
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1,padding=1)
        self.bn3 = nn.BatchNorm2d(64)

        # Adaptive pooling to make model input-size agnostic / dont want to use it for now
        # self.adaptive_pool = nn.AdaptiveAvgPool2d((4, 4))

        #size is determined by conv channels and the reduction in size by conv channels
        #channels * width * height because 256 /2 /2 /2 is 32
        self.fc1 = nn.Linear(in_features=64*32*32, out_features=128)
        self.fc2 = nn.Linear(in_features=128, out_features=self.numberOfClasses)

    def forward(self, x):
        #Block 1
        x = fun.relu(self.bn1(self.conv1(x)))
        x = fun.max_pool2d(x, kernel_size=2) # zmensovanie velkosti

        x = fun.relu(self.bn2(self.conv2(x)))
        x = fun.max_pool2d(x, kernel_size=2)

        x = fun.relu(self.bn3(self.conv3(x)))
        x = fun.max_pool2d(x, kernel_size=2)

        x = x.reshape(x.size(0), -1)

        x = fun.relu(self.fc1(x))
        x = self.fc2(x)
        return x


### Grad CAM Implementation
 -- On hold
 https://medium.com/@codetrade/grad-cam-in-pytorch-a-powerful-tool-for-visualize-explanations-from-deep-networks-bdc7caf0b282



## Model Training

### Training function

In [8]:
def train_model(model: nn.Module,
                train_loader: AnimalImageGenerator,
                val_loader: AnimalImageGenerator,
                criterion: nn.Module,
                optimizer: torch.optim.Optimizer,
                device: torch.device,
                epochs: int = 10,
                scheduler=None):

    model.to(device)

    for epoch in range(1, epochs+1):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        for images, labels in train_loader:
            images = torch.from_numpy(images).permute(0,3,1,2).float().to(device) # permute because the convolution layer gets input in the wrong order
            # images = torch.tensor(images, dtype=torch.float32, device=device)
            labels = torch.tensor(labels, dtype=torch.long, device=device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        train_loss = running_loss / total
        train_acc = correct / total

        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for images, labels in val_loader:
                images = torch.tensor(images, dtype=torch.float32, device=device)
                labels = torch.tensor(labels, dtype=torch.long, device=device)

                outputs = model(images)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * images.size(0)
                _, preds = torch.max(outputs, 1)
                val_correct += (preds == labels).sum().item()
                val_total += labels.size(0)

        val_loss /= val_total
        val_acc = val_correct / val_total

        if scheduler:
            scheduler.step(val_loss)

        print(f"Epoch {epoch}/{epochs} | "
              f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
        ## TODO: add f1 score

### Training

In [ ]:
img_class_model = ImageClassifier(NUM_CLASSES)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(img_class_model.parameters(), lr=0.001, momentum=0.9)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


train_model(model=img_class_model,
            train_loader=train_gen,
            val_loader=val_gen,
            criterion=criterion,
            optimizer=optimizer,
            device=device,
            epochs=10,
            scheduler=None)


### Hyper parameter sweeping